### Predicción de Noticias

In [1]:
import pickle
import seaborn as sns
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#### Lectura de Datos

In [3]:
with open("../dataset/data.pkl", "rb") as file:
    df = pickle.load(file)

In [89]:
def porcentajeNulos(df, umbral):
    # Calcula el porcentaje de nulos por columna

    porcentajeNulos = df.isna().sum().sort_values(ascending=False) / len(df)
    # Filtra las columnas que superan el umbral
    columnas = porcentajeNulos[(porcentajeNulos >= umbral)]

    # Muestra el resultado
    if not columnas.empty:
        print("Columnas con mas del {}% de valores nulos:".format(umbral * 100))
        print(columnas)
        
    else :
        print("No hay columnas con mas del {}% de valores nulos.".format(umbral * 100))

porcentajeNulos(df, umbral=0)

Columnas con mas del 0% de valores nulos:
label      0.0
content    0.0
dtype: float64


In [87]:
df.isna().sum().sort_values(ascending=False) / len(df)

label      0.0
content    0.0
dtype: float64